In [10]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt

import strawberryfields as sf
from strawberryfields import ops

%config InlineBackend.figure_format='retina'

At first we want to see how we can implement the $\hat{L}_z^2$ operation in the pennylane language. Just like in the example of the paper of Cindy Regal we will start off by modeling the number of $|\uparrow\rangle$ states and $|\downarrow\rangle$ states with Fock states. This is called "dual rail encoding".

We can write the $\hat{L}_z^2$ operator in exponential from with Kerr $K_i(\kappa)=e^{i\kappa n_i^2}$ and Cross-Kerr gates $KC_{i, j}(\kappa)=e^{i\kappa n_in_j}$. $\hat{L}_z^2 = (n_1 - n_2)^2 = n_1^2 - 2n_1n_2 + n_2^2$.

In [34]:
dev = qml.device('strawberryfields.fock', wires=2, cutoff_dim=11)

@qml.qnode(dev)
def Kerr_test(kappa, var=False):
    qml.FockState(5, wires=0)
    qml.FockState(0, wires=1)
    qml.Kerr(kappa, wires=0)
    qml.Kerr(kappa, wires=1)
    qml.CrossKerr(-kappa*2, wires=[0, 1])
    if var:  
        return [qml.var(qml.NumberOperator(0)),qml.var(qml.NumberOperator(1))]
    else:
        return [qml.expval(qml.NumberOperator(0)),qml.expval(qml.NumberOperator(1))]
    


In [35]:
Kerr_test(1)
print(Kerr_test.draw())

 0: ──|5⟩──Kerr(1)──╭CrossKerr(-2)──┤ ⟨n⟩ 
 1: ──|0⟩──Kerr(1)──╰CrossKerr(-2)──┤ ⟨n⟩ 



In [73]:
print(Kerr_test(1))


print("variance =", Kerr_test(1, var=True))

[5. 0.]
variance = [0. 0.]


In [37]:
dev = qml.device('strawberryfields.fock', wires=2, cutoff_dim=11)

@qml.qnode(dev)
def Kerr_beamsplitter(theta, kappa, var=False):
    qml.FockState(5, wires=0)
    qml.FockState(0, wires=1)
    
    qml.Beamsplitter(theta, 0, wires=[0, 1])
    qml.Kerr(kappa, wires=0)
    qml.Kerr(kappa, wires=1)
    qml.CrossKerr(-kappa*2, wires=[0, 1])
    if var:  
        return [qml.var(qml.NumberOperator(0)),qml.var(qml.NumberOperator(1))]
    else:
        return [qml.expval(qml.NumberOperator(0)),qml.expval(qml.NumberOperator(1))]
    



In [38]:
thetas = np.linspace(0, np.pi/2, 10)
print(list(Kerr_beamsplitter(1, theta) for theta in thetas))

[array([1.45963291, 3.54036709]), array([1.45963291, 3.54036709]), array([1.45963291, 3.54036709]), array([1.45963291, 3.54036709]), array([1.45963291, 3.54036709]), array([1.45963291, 3.54036709]), array([1.45963291, 3.54036709]), array([1.45963291, 3.54036709]), array([1.45963291, 3.54036709]), array([1.45963291, 3.54036709])]


Direct implementation in the stawberryfields plugin to extract samples

In [74]:
def circuit(theta, kappa, bs=False):
    prog = sf.Program(2)
    with prog.context as q:
        ops.Fock(5) | q[0]
        ops.Fock(0) | q[1]
        if bs:            
            ops.BSgate(theta, 0) | (q[0], q[1])
            
        ops.Kgate(kappa)   | q[0]
        ops.CKgate(kappa)  | (q[0], q[1])
        ops.Kgate(-2*kappa)| q[1]

        ops.MeasureFock()| q
    eng = sf.Engine("fock", backend_options={"cutoff_dim" :  10})
    result = eng.run(prog)
    return result



In [82]:
thetas = np.linspace(0 ,np.pi/2, 15)
result = np.array([circuit(i, 1).samples for i in thetas])
result_bs = np.array([circuit(i, 1, bs=True).samples for i in thetas])

In [83]:

print(result)
print("")
print(result_bs)

[[[5 0]]

 [[5 0]]

 [[5 0]]

 [[5 0]]

 [[5 0]]

 [[5 0]]

 [[5 0]]

 [[5 0]]

 [[5 0]]

 [[5 0]]

 [[5 0]]

 [[5 0]]

 [[5 0]]

 [[5 0]]

 [[5 0]]]

[[[5 0]]

 [[5 0]]

 [[5 0]]

 [[5 0]]

 [[3 2]]

 [[3 2]]

 [[3 2]]

 [[5 0]]

 [[2 3]]

 [[2 3]]

 [[0 5]]

 [[1 4]]

 [[0 5]]

 [[0 5]]

 [[0 5]]]


In [85]:
thetas = np.linspace(0 ,np.pi/2, 15)
result = [circuit(i, 0).samples for i in thetas]